In [2]:
from sagemaker.sklearn.estimator import SKLearn
import sagemaker
import boto3

FRAMEWORK_VERSION = "0.23-1"


# Configure the SKLearn estimator
sklearn_estimator = SKLearn(
    entry_point="script.py",
    role="arn:aws:iam::187704205293:role/sagemaker",
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="LR-custom-sklearn",
    hyperparameters={
        "l1_ratio": 0,
    },
    use_spot_instances=True,
    max_wait=7200,
    max_run=3600
)

# Create a SageMaker session
sess = sagemaker.Session()

# Specify the S3 bucket and prefix
bucket = 'ice-cream-reviews-sagemakerbucket'
print('Using bucket ' + bucket)

sk_prefix = "sagemaker/ice-cream-reviews-sagemakerbucket/sklearncontainer"
trainpath = "s3://ice-cream-reviews-sagemakerbucket/sagemaker/ice-cream-reviews/sklearncontainer/train.csv"
testpath = "s3://ice-cream-reviews-sagemakerbucket/sagemaker/ice-cream-reviews/sklearncontainer/test.csv"

# Launch the training job with asynchronous call
sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait=True)

Using bucket ice-cream-reviews-sagemakerbucket
Using provided s3_resource


INFO:sagemaker:Creating training-job with name: LR-custom-sklearn-2023-07-14-22-43-01-688


2023-07-14 22:43:03 Starting - Starting the training job...
2023-07-14 22:43:18 Starting - Preparing the instances for training.........
2023-07-14 22:45:05 Downloading - Downloading input data...
2023-07-14 22:45:35 Training - Downloading the training image...
2023-07-14 22:46:06 Training - Training image download completed. Training in progress...2023-07-14 22:46:10,250 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2023-07-14 22:46:10,253 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-07-14 22:46:10,292 sagemaker_sklearn_container.training INFO     Invoking user training script.
2023-07-14 22:46:10,454 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-07-14 22:46:10,465 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-07-14 22:46:10,478 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-07-14

In [3]:
sklearn_estimator.latest_training_job.wait(logs="None")
sm_boto3 = boto3.client("sagemaker")
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifact persisted at " + artifact)


2023-07-14 22:46:32 Starting - Preparing the instances for training
2023-07-14 22:46:32 Downloading - Downloading input data
2023-07-14 22:46:32 Training - Training image download completed. Training in progress.
2023-07-14 22:46:32 Uploading - Uploading generated training model
2023-07-14 22:46:32 Completed - Training job completed

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials



Model artifact persisted at s3://sagemaker-us-east-1-187704205293/LR-custom-sklearn-2023-07-14-22-43-01-688/output/model.tar.gz


In [4]:
artifact

's3://sagemaker-us-east-1-187704205293/LR-custom-sklearn-2023-07-14-22-43-01-688/output/model.tar.gz'

In [5]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name =  model_name,
    model_data=artifact,
    role="arn:aws:iam::187704205293:role/sagemaker",
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION,
)

In [6]:
model_name

'Custom-sklearn-model-2023-07-14-22-46-58'

In [7]:
# Endpoint deployment
endpoint_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name,
)

EndpointName=Custom-sklearn-model-2023-07-14-22-46-58


INFO:sagemaker:Creating model with name: Custom-sklearn-model-2023-07-14-22-46-58
INFO:sagemaker:Creating endpoint-config with name Custom-sklearn-model-2023-07-14-22-46-58
INFO:sagemaker:Creating endpoint with name Custom-sklearn-model-2023-07-14-22-46-58


------!

In [8]:
endpoint_name

'Custom-sklearn-model-2023-07-14-22-46-58'

In [15]:
sm_boto3.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': 'e7196d2a-fb62-4980-b302-fce1c2a25820',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e7196d2a-fb62-4980-b302-fce1c2a25820',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Fri, 14 Jul 2023 22:58:31 GMT'},
  'RetryAttempts': 0}}